In [1]:
import cv2
import pandas as pd
from utils import *
from gNet import GestureNet
import tensorflow as tf
from tensorflow import keras
from keras import Sequential, Input
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
from datetime import datetime
from sklearn.model_selection import train_test_split
import pandas as pd

(      0_x  0_y       1_x       1_y       2_x       2_y       3_x       3_y   
0     0.0  0.0  0.225734 -0.022573  0.363431 -0.051919  0.453725 -0.063205  \
1     0.0  0.0  0.222727 -0.018182  0.365909 -0.040909  0.461364 -0.059091   
2     0.0  0.0  0.219178 -0.013699  0.356164 -0.029680  0.449772 -0.043379   
3     0.0  0.0  0.244804 -0.030023  0.383372 -0.046189  0.480370 -0.060046   
4     0.0  0.0  0.239726 -0.015982  0.376712 -0.029680  0.470320 -0.041096   
...   ...  ...       ...       ...       ...       ...       ...       ...   
6175  0.0  0.0  0.153684 -0.084211  0.240000 -0.242105  0.240000 -0.385263   
6176  0.0  0.0  0.134454 -0.117647  0.178571 -0.281513  0.151261 -0.413866   
6177  0.0  0.0  0.132632 -0.120000  0.178947 -0.284211  0.149474 -0.412632   
6178  0.0  0.0  0.132353 -0.121849  0.176471 -0.287815  0.147059 -0.418067   
6179  0.0  0.0  0.128151 -0.123950  0.172269 -0.287815  0.140756 -0.415966   

           4_x       4_y  ...      16_x      16_y      17_x   

In [5]:
batch_size = 4
epoch = 100

# backup_dir = Path('./model/') / f"{int(datetime.utcnow().timestamp())}"
# backup_dir = Path('./model/') / "1700691728" #95.71
# backup_dir = Path('./model/') / "1700693117" #95.95
backup_dir = Path('./model/') / "primary"
ckpt_location = backup_dir / "model_ckpt.ckpt"
output_location = backup_dir / "model_output"


ckpt_backup_callback = tf.keras.callbacks.ModelCheckpoint(filepath=ckpt_location, save_weights_only=True, verbose=1)

label = getLabel()
X, Y = getTrain2DF()
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, stratify=Y)
args = getGNetArgs()
gnet = GestureNet(args=args)
model = gnet.model()

Model: "GestureNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 48)                2064      
                                                                 
 batch_normalization_4 (Batc  (None, 48)               192       
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 48)                0         
                                                                 
 dropout_4 (Dropout)         (None, 48)                0         
                                                                 
 dense_7 (Dense)             (None, 24)                1176      
                                                                 
 batch_normalization_5 (Batc  (None, 24)               96        
 hNormalization)                                        

## Train model

In [6]:
model.fit(x_train, y_train, epochs=epoch, callbacks=[ckpt_backup_callback])
model.save_weights(output_location)

Epoch 1/100
145/155 [===========================>..] - ETA: 0s - loss: 1.0027 - accuracy: 0.6817
Epoch 1: saving model to model\primary\model_ckpt.ckpt
155/155 [==============================] - 3s 3ms/step - loss: 0.9806 - accuracy: 0.6913
Epoch 2/100
142/155 [==========================>...] - ETA: 0s - loss: 0.4813 - accuracy: 0.8774
Epoch 2: saving model to model\primary\model_ckpt.ckpt
155/155 [==============================] - 0s 3ms/step - loss: 0.4760 - accuracy: 0.8788
Epoch 3/100
147/155 [===========================>..] - ETA: 0s - loss: 0.3448 - accuracy: 0.9069
Epoch 3: saving model to model\primary\model_ckpt.ckpt
155/155 [==============================] - 0s 3ms/step - loss: 0.3387 - accuracy: 0.9090
Epoch 4/100
134/155 [========================>.....] - ETA: 0s - loss: 0.2835 - accuracy: 0.9172
Epoch 4: saving model to model\primary\model_ckpt.ckpt
155/155 [==============================] - 0s 3ms/step - loss: 0.2744 - accuracy: 0.9187
Epoch 5/100
141/155 [===============

## Load model

In [7]:
model.load_weights(output_location)
loss, acc = model.evaluate(x_test, y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

39/39 - 0s - loss: 0.0108 - accuracy: 0.9960 - 276ms/epoch - 7ms/step
Restored model, accuracy: 99.60%


In [8]:
law = [0.0, 0.0, -0.08523409363745499, 0.004801920768307323, -0.16566626650660263, -0.027611044417767107, -0.21728691476590636, -0.062424969987995196, -0.2665066026410564, -0.08883553421368548, -0.15966386554621848, -0.16326530612244897, -0.22448979591836735, -0.22809123649459784, -0.2689075630252101, -0.2653061224489796, -0.3097238895558223, -0.29411764705882354, -0.1092436974789916, -0.19447779111644659, -0.15846338535414164, -0.2809123649459784, -0.1908763505402161, -0.33733493397358943, -0.22088835534213686, -0.3865546218487395, -0.05042016806722689, -0.20048019207683074, -0.08283313325330131, -0.2917166866746699, -0.09963985594237695, -0.3517406962785114, -0.11764705882352941, -0.4069627851140456, 0.013205282112845138, -0.18847539015606243, 0.004801920768307323, -0.2581032412965186, -0.003601440576230492, -0.304921968787515, -0.015606242496998799, -0.34933973589435774]
df = pd.DataFrame(law).T
print(df.shape)
pred = model.predict(df)

print(pred)


(1, 42)
1/1 [==============================] - 0s 185ms/step
[[2.3589057e-13 2.4834568e-11 9.9994338e-01 1.9347189e-08 7.8898249e-10
  5.6561381e-05]]
